In [3]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import numpy as np
from scipy.interpolate import interp2d
from matplotlib.ticker import FuncFormatter
import h5py
import ijson

In [1]:
def generate_data(file_path):
    with open(file_path) as f:
        for line in f:
            if not line.strip():
                continue

            json_obj = json.loads(line)
            yield json_obj
    
def plot_json_data(json_file, color = 'k', label = ''):

    all_waves = []
    times = []
    data = []
    
    with open(json_file) as f:
        for line in f:
            if not line.strip():
                continue
            # Load each line as a JSON object and append it to the list
            json_obj = json.loads(line)
            data.append(json_obj)
        
    # with open(json_file, "rb") as f:

        # for amps in ijson.items(f, "amps"):
        #     all_waves.append(amps[0])
        # for time in ijson.items(f, "time"):
        #     times.append(time)

    
    
    # with open(json_file, 'r') as file:
        # lines = file.readlines()

    # total_amp = []
    # times = []
    # all_waves = []
    # # Process each JSON object in the file
    # for line in lines:
    #     try:
    #         data = json.loads(line)

    #         # Access the 'amps' and 'time' keys in the JSON data
    #         amps = data.get('amps', [])
    #         all_waves.append(amps[0])
    #         times.append(data.get('time', []))
    #         # for n,d in enumerate(amps):
    #         #     total_amp.append(np.dot(d,d)) 
                
    #         # for t in list(time):
    #         #     times.append(time)
    #         # # Process or print the data as needed
    #         # print("Amps Data:", amps)
    #         # print("Time Data:", time_data)
            
    #     except json.JSONDecodeError as e:
    #         print(f"Error decoding JSON: {e}")
    return data
    
def format_ticks(x, pos):
    return '{:.1f}'.format(x / 2.0)
    
    
def plot_images(amp):
    cmap = 'coolwarm' #colormap for figure heatmaps
    rows = 1
    cols = 1
    fig1 = figure(figsize=(12,6))
    fig1.subplots_adjust(hspace = 0.03, wspace = 0.1)
    gs = GridSpec(rows,cols+1)
    ax1 = fig1.add_subplot(gs[:,0])
    
    print(amp)
    plotn = 0
    plotm = 0
    x_dim = 30+1
    y_dim = 80+1
    vmin = 0.004
    vmax = 0.02
    
    # Reshape the array to x by y
    waves_reshaped = np.array(amp)[:x_dim * y_dim].reshape((x_dim, y_dim))
    im = plt.imshow(waves_reshaped, cmap = cmap, vmin = vmin, vmax = vmax)

    xticks = ax1.get_xticks()
    yticks = ax1.get_yticks()
    ax1.xaxis.set_major_formatter(FuncFormatter(format_ticks))
    ax1.yaxis.set_major_formatter(FuncFormatter(format_ticks))  
    ax1.set_xlabel('Y-axis (mm)')
    ax1.set_ylabel('Z-axis (mm)')
    ax1.set_title('Transmit Time ($\mathregular{\mu}$s), 50 MHz TR')

# This function utilizes a short term avg long term avg (STA/LTA) detection algorithm to determine wave transmit time
def classic_sta_lta(wave, nsta = None, nlta = None):  
    df_wave = pd.DataFrame(wave)
    lta = df_wave.rolling(window=nlta).mean()
    sta = df_wave.rolling(window=nsta).mean()
    tiny = np.finfo(0.0).tiny
    lta[lta < tiny] = tiny
    return (sta / lta, sta, lta)
    
def first_break_picker(waves,t_0,t_1,lta2sta):
    index = []
    stalta_ = []
    points = len(waves[0])
    t_sta = 0.5E-6 
    sampling_rate = 1/abs((1./points)*(t_1-t_0)*10E-6) #spacing of data collection for one wave
    nsta = int(t_sta * sampling_rate)
    nlta = lta2sta * nsta
    for wave in waves:
        stalta,__,__ = classic_sta_lta(wave**2,nsta=nsta,nlta=nlta)
        thres = 0.75 * np.nanmax(stalta) #0.75 arbitrarily set
 
        stalta_.append(stalta)
        stalta = stalta.values #convert to numpy array for argmax finder
        
        '''Threshold'''
        max_index = (stalta > thres).argmax() if (stalta > thres).any() else -1
        # max_index = stalta.idxmax()
        index.append((float(max_index)/points)*(t_1 - t_0) + t_0)
        
    return stalta_,index
    
def convert_json_to_hdf5(json_filename, hdf5_filename):
    with open(json_filename, 'r') as json_file:
        data = json.load(json_file)

    with h5py.File(hdf5_filename, 'w') as hdf5_file:
        hdf5_file.create_dataset('amps', data=data.get('amps', []))
        hdf5_file.create_dataset('time', data=data.get('time', []))

def analyze_hdf5(hdf5_filename):
    with h5py.File(hdf5_filename, 'r') as hdf5_file:
        waves = np.array(hdf5_file['amps'])
        times = np.array(hdf5_file['time'])
        
        for n,d in enumerate(waves):
            amp.append(np.dot(d,d)) 
            
        # Plotting
        # plt.plot(times, waves)
        plt.xlabel('Time')
        plt.ylabel('Waves')
        plt.title('Wave Data from HDF5')
        plt.show()

if __name__ == "__main__":
    # json_file_path = "files/ultrasound/WC_20240126_Y70-Z25_AT-LMFP_3.json"
    json_file_path = "/home/jovyan/work/General/Group/Data/Ultrasound/20240202_WC_JE_EFC2.json"
    # data = plot_json_data(json_file_path)
    for json_obj in generate_data(json_file_path):
        process_data(json_obj)
            
    # hdf5_file_path = "20240202_WC_JE_EFC2.h5"
    
    # Convert JSON to HDF5
    # convert_json_to_hdf5(json_file_path, hdf5_file_path)
    
    # Analyze HDF5 data
    # analyze_hdf5(hdf5_file_path)

    amp = []
    
    # for n,d in enumerate(waves):
    #     amp.append(np.dot(d,d)) 
        
    # plt.plot(((np.array(times))-(np.array(times))[0])/3600., (np.array(amp))/max(amp), 'k', lw = 0.8)
    # plt.xlim(0.8,7)
    # plt.xlabel('Time (hrs)')
    # plt.ylabel('Total Amplitude (A/A$\mathregular{_0}$)')
    # plot_images(amp)
    
    # delay = 14.
    # duration = 2. 
    # tofs = np.linspace(duration, duration + delay, len(waves))

    # lta2sta = 12
    # ref = 2

    # stalta_,index = first_break_picker(np.array(waves),14.,16.,lta2sta)
    # print(index)
    # plot_images(index)
    
    showme(dpi=400)
    

    
    

NameError: name 'json' is not defined